In [ ]:
# Ensure directory exists
os.makedirs('../models/data', exist_ok=True)

with open('../models/data/rf_dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)

print("Dataset saved to models/data/rf_dataset.pkl")

## 5. Save Dataset
Save the generated dataset to a file for use in the training notebook.

In [ ]:
# Main Data Generation Loop
dataset = {'data': [], 'labels': [], 'snr': []}

for mod in MODULATIONS:
    print(f"Processing {mod}...")
    for snr in SNR_RANGE:
        for _ in range(NUM_SIGNALS):
            # 1. Generate Clean Signal
            sig = generate_signal(mod, NUM_SAMPLES)
            
            # 2. Apply Channel (AWGN)
            sig_noisy = apply_awgn(sig, snr)
            
            # 3. Store as (2, 1024) array for NCHW or (1024, 2) for NWC
            # Using (1024, 2) here: [Real, Imag]
            sig_formatted = np.stack([sig_noisy.real, sig_noisy.imag], axis=1)
            
            dataset['data'].append(sig_formatted)
            dataset['labels'].append(mod)
            dataset['snr'].append(snr)

# Convert to numpy arrays
dataset['data'] = np.array(dataset['data'])
dataset['labels'] = np.array(dataset['labels'])
dataset['snr'] = np.array(dataset['snr'])

print(f"Dataset generated. Shape: {dataset['data'].shape}")

In [ ]:
def apply_awgn(signal, snr_db):
    """Applies AWGN to the signal for a given SNR (dB)."""
    # Calculate signal power
    s_p = np.mean(np.abs(signal)**2)
    
    # Calculate noise power based on SNR
    snr_linear = 10**(snr_db / 10.0)
    n_p = s_p / snr_linear
    
    # Generate complex noise
    noise = (np.random.randn(len(signal)) + 1j * np.random.randn(len(signal))) * np.sqrt(n_p / 2)
    
    return signal + noise

## 3. Channel Impairments
Add Additive White Gaussian Noise (AWGN) to simulating transmission over a channel.

In [ ]:
def generate_signal(mod_type, num_samples):
    """Generates a random baseband signal for a given modulation type."""
    M = MODULATIONS[mod_type]
    
    # Generate random symbols
    symbols = np.random.randint(0, M, num_samples)
    
    if mod_type == 'BPSK':
        iq = np.exp(1j * (2 * np.pi * symbols / M))
    elif mod_type == 'QPSK':
        iq = np.exp(1j * (2 * np.pi * symbols / M + np.pi/4))
    elif mod_type == '8PSK':
        iq = np.exp(1j * (2 * np.pi * symbols / M))
    elif 'QAM' in mod_type:
        # Simple rectangular QAM generation (simplified)
        m_sqrt = int(np.sqrt(M))
        real = 2 * np.random.randint(0, m_sqrt, num_samples) - (m_sqrt - 1)
        imag = 2 * np.random.randint(0, m_sqrt, num_samples) - (m_sqrt - 1)
        iq = real + 1j * imag
        # Normalize energy
        iq = iq / np.sqrt(np.mean(np.abs(iq)**2))
        
    return iq

## 2. Modulation Functions
Implement helper functions to generate baseband IQ signals for each modulation scheme.

In [ ]:
# Simulation Parameters
NUM_SAMPLES = 1024  # Length of each IQ sequence
NUM_SIGNALS = 1000  # Number of signals per modulation per SNR level
SNR_RANGE = range(-20, 20, 2) # SNR levels in dB

# Supported Modulations
MODULATIONS = {
    'BPSK': 2,
    'QPSK': 4,
    '8PSK': 8,
    '16QAM': 16,
    '64QAM': 64
}

print(f"Generating dataset for {len(MODULATIONS)} modulation types across {len(SNR_RANGE)} SNR levels.")

## 1. Parameters Configuration
Define the parameters for signal generation, including sample rate, number of samples per signal, and modulation types.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pickle
import os

# Set random seed for reproducibility
np.random.seed(42)